<a href="https://colab.research.google.com/github/evinracher/3010090-ontological-engineering/blob/main/week3/part2/05_BD_vectores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01 - Cargadores de documentos con LangChain 1.0 y Gemini 🇨🇴

En este cuaderno trabajaremos el tema de **conectores de datos** en el contexto de **LangChain 1.0** usando como LLM **Gemini**.

Verás los siguientes temas básicos, listos para ejecutar en **Google Colab**:

1. Instalación de dependencias para LangChain 1.0 + Gemini  
2. Configuración de la API Key de Gemini  
3. Cargadores de documentos (PDF como caso base)  
4. Transformación de documentos: división en *chunks*  
5. Creación de *embeddings* con Gemini  
6. Creación de una base vectorial (FAISS)  
7. Cadena de pregunta-respuesta (RAG) sobre el PDF usando Gemini  

> **Nota:** Todos los ejemplos están simplificados para que sirvan como plantilla didáctica.


## 1. Instalación de librerías necesarias

In [1]:
# Si estás en Google Colab, ejecuta esta celda primero.
# Puede tardar un poco.

!pip -q install -U langchain-core langchain-community langchain-text-splitters google-generativeai langchain-google-genai fastembed faiss-cpu pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 501.4/501.4 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.0/331.0 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.8/324.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently ta

## 2. Configuración de la clave de API de Gemini

In [2]:
from google.colab import userdata
import os

# Obtener la API key desde userdata
api_key = userdata.get('GOOGLE_API_KEY')

# Opcional: Guardarla como variable de entorno
os.environ['GOOGLE_API_KEY'] = api_key

# Verificar que se haya cargado correctamente
print("API Key cargada:", "Sí" if api_key else "No")
print("Primeros caracteres:", api_key[:10] if api_key else "No encontrada")

API Key cargada: Sí
Primeros caracteres: AIzaSyDx4e


## 3. Importaciones base: LangChain 1.0 + Gemini

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings

print("✅ Librerías importadas correctamente.")

✅ Librerías importadas correctamente.


## 4. Cargadores de documentos: ejemplo con PDF

En este ejemplo vamos a:

1. Subir un archivo PDF a Colab  
2. Cargarlo con `PyPDFLoader`  
3. Ver parte de su contenido

> Puedes usar cualquier PDF que tengas a mano (artículo, paper, capítulo de libro, etc.).


In [4]:
# 4.1. Subir el archivo PDF desde tu equipo
# Ejecuta esta celda y selecciona un archivo .pdf

from google.colab import files

upload = files.upload()
pdf_path = list(upload.keys())[0]
print(f"📄 Archivo cargado: {pdf_path}")

Saving Documento tecnologías emergentes.pdf to Documento tecnologías emergentes.pdf
📄 Archivo cargado: Documento tecnologías emergentes.pdf


In [5]:
# 4.2. Cargar el PDF usando PyPDFLoader
loader = PyPDFLoader(pdf_path)
docs = loader.load()

print(f"✅ Páginas cargadas: {len(docs)}")
print("\n📄 Vista previa del contenido de la primera página:\n")
print(docs[0].page_content[:1000])

✅ Páginas cargadas: 4

📄 Vista previa del contenido de la primera página:

Estas son las 9 tecnologías 
emergentes para el próximo 
2025 
  
“Que la tecnología ha cambiado nuestra manera de vivir e interactuar 
es un hecho. Sin embargo, aún no somos conscientes de las 
potencialidades de usos de las tecnologías.Por ejemplo, para el año 
2025 se espera una verdadera revolución tecnológica, sobre todo 
enfocado en el sector bio-médico pero también en las relaciones 
humanas entre individuos a distancia, en la protección del medio 
ambiente o en la protección de nuestros datos personales”, afirma 
Juan Quintanilla, director general de Syntonize. 
9 Tecnologías emergentes según Syntonize 
La aplicación de nuevas tecnologías que hagan más fácil la vida a 
profesionales, estudiantes, mayores, empresas o instituciones 
públicas se espera que aumente en los próximos años. Entre ellas se 
encuentran; 
 Producción optimizada por la Inteligencia Artificial: las 
empresas están adoptando rápidame

## 5. Transformación de documentos: división en *chunks*

Para usar un LLM de forma eficiente y también para crear una base vectorial,
es conveniente dividir el documento en fragmentos (*chunks*).

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,      # longitud máxima de cada chunk
    chunk_overlap=100,   # solapamiento entre chunks para mantener contexto
)

chunks = text_splitter.split_documents(docs)
print(f"✅ Número de chunks generados: {len(chunks)}")
print("\n📄 Ejemplo de un chunk:\n")
print(chunks[0].page_content[:500])

✅ Número de chunks generados: 9

📄 Ejemplo de un chunk:

Estas son las 9 tecnologías 
emergentes para el próximo 
2025 
  
“Que la tecnología ha cambiado nuestra manera de vivir e interactuar 
es un hecho. Sin embargo, aún no somos conscientes de las 
potencialidades de usos de las tecnologías.Por ejemplo, para el año 
2025 se espera una verdadera revolución tecnológica, sobre todo 
enfocado en el sector bio-médico pero también en las relaciones 
humanas entre individuos a distancia, en la protección del medio 
ambiente o en la protección de nuestros 


## 6. Incrustaciones de texto (Embeddings) con Gemini

Usaremos el modelo `BAAI/bge-small-en-v1.5` de FastEmbedEmbeddings para convertir cada chunk
en un vector numérico de alta dimensión.

In [7]:
embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-small-en-v1.5")

# Probar con un texto simple
vector_ejemplo = embeddings.embed_query("La inteligencia artificial está transformando la educación.")
print(f"Dimensión del vector de ejemplo: {len(vector_ejemplo)}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Dimensión del vector de ejemplo: 384


## 7. Creación de una base vectorial con FAISS

Ahora creamos una base de datos vectorial a partir de los *chunks* usando FAISS.
Esto permitirá hacer **búsqueda semántica** sobre el contenido del PDF.

In [8]:
vectorstore = FAISS.from_documents(chunks, embeddings) # Se guarda el texto y la representación semántica
retriever = vectorstore.as_retriever(search_kwargs={"k": 4}) # Primeros 4 resultados

print("✅ Base vectorial creada y retriever configurado.")

✅ Base vectorial creada y retriever configurado.


## 8. Definir el modelo de chat Gemini

In [9]:
llm = ChatGoogleGenerativeAI(
    model="models/gemini-2.5-flash-lite",
    temperature=0.3,
)

print("✅ Modelo Gemini configurado: models/gemini-2.5-flash-lite")

✅ Modelo Gemini configurado: models/gemini-2.5-flash-lite


## 9. Cadena de Pregunta-Respuesta (RAG) sobre el PDF

Usaremos el patrón LCEL de LangChain 1.0 para construir una cadena que:

1. Recupera los chunks más relevantes vía `retriever`  
2. Forma un prompt con contexto + pregunta del usuario  
3. Envía el prompt a Gemini  
4. Devuelve una respuesta en lenguaje natural


In [10]:
prompt = ChatPromptTemplate.from_template("""
Eres un asistente experto en comprensión de documentos.

Usando exclusivamente la siguiente información de contexto, responde de forma clara,
estructurada y en español a la pregunta del usuario.

Contexto:
{context}

Pregunta:
{question}
""")

# 1) Definimos un paso que obtiene contexto desde el retriever
def get_context(query: str):
    docs_relacionados = retriever.invoke(query)   # << CORRECTO EN LC 1.0
    return "\n\n".join([d.page_content for d in docs_relacionados])


# 2) Construimos la cadena con LCEL
rag_chain = (
    RunnableParallel(
        context=lambda x: get_context(x["question"]),
        question=RunnablePassthrough()
    )
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ Cadena RAG construida correctamente.")

✅ Cadena RAG construida correctamente.


## 10. Probar la cadena RAG con tus propias preguntas

Ahora puedes hacer preguntas en lenguaje natural sobre el contenido del PDF cargado.


In [11]:
pregunta = "¿Cuál es el tema principal del documento y cuáles son sus conclusiones más importantes?"

respuesta = rag_chain.invoke({"question": pregunta})

print("🧠 Pregunta:", pregunta)
print("\n📌 Respuesta de Gemini:\n")
print(respuesta)

🧠 Pregunta: ¿Cuál es el tema principal del documento y cuáles son sus conclusiones más importantes?

📌 Respuesta de Gemini:

El documento aborda las transformaciones tecnológicas esperadas en los próximos años y sus impactos en diversos ámbitos.

Las conclusiones más importantes son:

*   **Avances en la comprensión de patógenos:** La tecnología permitirá acelerar el muestreo, digitalización e interpretación de datos de microbiomas, lo que transformará la comprensión de la propagación de patógenos.
*   **Privacidad como prioridad:** La privacidad será generalizada y priorizada, considerando la protección y control de activos de datos confidenciales como la norma. Las tecnologías de mejora de la privacidad se convertirán en una categoría tecnológica propia y un elemento fundamental en las estrategias empresariales.
*   **Aumento de la aplicación de nuevas tecnologías:** Se espera un incremento en la aplicación de tecnologías que faciliten la vida de profesionales, estudiantes, mayores, 

Al hacer una pregunta más específica:

In [12]:
pregunta = "¿A qué se refieren con 'La privacidad estará generalizada y priorizada'"

respuesta = rag_chain.invoke({"question": pregunta})

print("🧠 Pregunta:", pregunta)
print("\n📌 Respuesta de Gemini:\n")
print(respuesta)

🧠 Pregunta: ¿A qué se refieren con 'La privacidad estará generalizada y priorizada'

📌 Respuesta de Gemini:

Con "La privacidad estará generalizada y priorizada" se refieren a que la capacidad de los consumidores para proteger y controlar sus activos de datos confidenciales se convertirá en la norma, no en la excepción. Las tecnologías de mejora de la privacidad se consolidarán como una categoría tecnológica propia y serán un componente esencial en las estrategias de privacidad y seguridad de las empresas.


---

### ✅ Resumen de este cuaderno

En este notebook vimos:

- Cómo configurar **Gemini** en Google Colab usando `langchain-google-genai`.  
- Cómo cargar un **PDF** con `PyPDFLoader`.  
- Cómo dividir el documento en **chunks** con `RecursiveCharacterTextSplitter`.  
- Cómo generar **embeddings** con `GoogleGenerativeAIEmbeddings`.  
- Cómo construir una base vectorial con **FAISS**.  
- Cómo implementar una cadena **RAG** con LCEL para hacer preguntas sobre el PDF.

Este cuaderno sirve como plantilla base para extensiones posteriores: integración con más tipos de documentos,
uso de otros almacenes vectoriales, o construcción de interfaces (por ejemplo, Gradio o Streamlit).


In [13]:
!pip freeze > 05_BD_vectores_requirements.txt